# Forrestania: Gravity & Magnetics (Pure Python)

```{figure} ./images/landing.png
---
scale: 30%
align: right
---
```

This case study focuses on the standalone and joint inversion of ground gravity and airborne magnetic data, all using open-source libraries. It uses public data downloaded from the [Geological Survey of Western Australia](https://portal.ga.gov.au/persona/gadds) over the Forrestania geological province. We cover the following steps

- [Data imports and pre-processing](forrestania-data-os)
- [Standalone inversion of gravity data](forrestania-gravity-os)
- [Standalone inversion of magnetic data](forrestania-magnetics-os)
- [Joint cross-gradient inversion](forrestania-joint-os)
- [K-means clustering analysis](forrestania-kmeans-os)

## Geological setting

To be completed...

(forrestania-data-os)=
## Data

[Download the data package here](https://github.com/MiraGeoscience/simpeg-drivers/raw/refs/heads/GEOPY-2297/simpeg_drivers-assets/Case%20studies/Forrestania_SRTM1%20Australia_MGA50_CSV.zip?download=)

We first need to import the data into a usable format.

 - Airborne magnetic survey: `60472_AOI4.csv`
 - Ground gravity survey: `Forrestania_Gravity_Station_trim_.csv`
 - Digital Elevation Model (DEM): `Forrestania_SRTM1 Australia_MGA50.tiff`

In [2]:
import geoh5y as gp
import numpy as np

ModuleNotFoundError: No module named 'geoh5y'

### Processing

Note that both the DEM and airborne magnetic survey were imported at an arbitrary 0 m elevation and do not reflect the true position.
We first need to modify our object such that the data is accurately located in 3D space. 

#### Step 1: Convert the 2D DEM into a 3D surface
```{figure} ./images/create_surface.png
---
scale: 10%
---
[Click to enlarge]
```

#### Step 2a: Transfer the DEM data onto the airborne survey (curve). 
```{figure} ./images/transfer_dem.png
---
scale: 10%
---
[Click to enlarge]
```

#### Step 2b: Re-assign the Z channel to the DEM + radar height.
```{figure} ./images/drape_mag.png
---
scale: 10%
---
[Click to enlarge]
```


(forrestania-gravity-os)=
## Gravity

We begin our analysis with the ground gravity survey. We will attempt to estimate the shape and depth of the causative body at depth.

### Inputs

Gravity data are sensitive to variations in the density of rocks. The data were already processed and provided as terrain-corrected (2.67 g/cc) residual data. We can therefore go directly to the inversion phase. 

```{figure} ./images/gravity_data.png
```

### Inversion options

The following options are set for the inversion.

```{figure} ./images/gravity_setup.png
---
scale: 10%
---
[Click to enlarge]
```
- Assign the data uncertainty to a quarter of the standard deviation (0.05 mGal). This is a somewhat conservative estimate that takes into account both the instrumental and numerical noise.
- Use a `0 g/cc` reference value for the model. This assumes that the signal from the background density was accurately subtracted from the data.
- Leave the `Mesh` option unchecked. We will let the code create an octree mesh based on the average data separation and extent.

### Results

After completion of the inversion, we load the results and assess convergence and data fit.

```{figure} ./images/grav_predicted.png
```

We note the following:

 - It took 13 iterations to converge to the target misfit.
 - The following 12 iterations have increased the model complexity ($phi_m$) but preserved the data fit ($phi_d$).
 - Most of the important gravity anomalies appear to be reproduced by the final model.

#### Smooth solution (L2-norm)

```{figure} ./images/grav_l2_model.png
```

The smooth solution (iteration 13) indicates the presence of a dense anomaly extending at depth with density contrasts ranging from [-0.02, 0.05] g/cc. The negative density contrasts appear to be localized around the main positive anomaly, likely due to the smoothness constraint and the lack of `bound` constraints. Some smaller "fuzzy" anomalies are visible at the margins.  

#### Compact solution (L0-norm)

```{figure} ./images/grav_lp_model.png
```

As an alternative solution, the compact model (iteration 25) recovers a well-defined dense body within a mostly uniform background. Density contrasts have substantially increased in the range of [0.0, 0.56] g/cc.

<p style="page-break-after:always;"></p>

(forrestania-magnetics-os)=
## Magnetics

We follow up with the processing of the airborne magnetic data.

### Inputs

The source of the magnetic signal can generally be attributed to the presence or destruction of magnetic minerals in rocks (mainly magnetite). Magnetometers measure the Total Magnetic Intensity (TMI), which includes both the primary (source) field and secondary fields (signal) from the local geology. 

#### Step 1: Residual data

The inversion routine requires Residual Magnetic Intensity (RMI). The first step is to compute and remove the primary field (IGRF) at the time and location of acquisition.  The survey was conducted between February and March 1988.

##### Step 1a: Set the coordinate system of the survey

Before computing the IGRF, we need to locate the survey in world coordinates. This specific project is referenced to the `MGA zone 50` coordinate system.

```{figure} ./images/mag_coordinates.png
---
scale: 10%
---
[Click to enlarge]
```

##### Step 1b: Lookup/remove IGRF 

```{figure} ./images/mag_igrf.png
---
scale: 10%
---
[Click to enlarge]
```

The **inducing field parameters** at the time and location of the survey are

- Magnitude: 59127 nT
- Inclination: -66.9$\degree$
- Declination: -1.05$\degree$  

#### Step 2: Detrend
The local background field appears to be slightly lower (~112 nT) than the computed IGRF model, as most of the data away from the main anomaly are below 0. To avoid modelling this background trend, we can remove the median value. 

```{figure} ./images/mag_detrend.png
---
scale: 10%
---
[Click to enlarge]
```

#### Step 3: Downsampling

We can downsample the data along lines to reduce the computation cost of the inversion. Since the average flight height of this survey was 60 m, we can confidently downsample the lines by the same amount without affecting the wavelengths contained in the data.

```{figure} ./images/mag_downsample.png
---
scale: 10%
---
[Click to enlarge]
```

### Create a mesh

This time, we will create a mesh using the [Octree Mesh Creation](mesh-design) application. In preparation for the joint inversion, we will simply modify the gravity mesh parameters. By doing so, we guarantee that the extent and base cell size remain the same, only the refinements will be optimized to the magnetic survey.

```{figure} ./images/mag_mesh.png
```

#### Step 1: Load the octree parameters
Inspect the files attached to the gravity mesh object and load the `ui.json`.

#### Step 2: Modify the refinements
Change the input object for `Refinement A` for the curve object of the magnetic survey.

#### Step 3: Run and load the results

 
### Inversion options

The following options are set for the inversion.

```{figure} ./images/mag_setup.png
---
scale: 10%
---
[Click to enlarge]
```

- Set the inducing field parameters as described above.
- Assign the data uncertainty to a quarter of the standard deviation (125 nT).
- Use a `0 SI` reference value for the model. This assumes that we removed all background signal from the data.
- Assign the newly created octree mesh object.
- Leave the default lower bound value to 0 SI (non-negative). 

### Results

After completion of the inversion, we load the results and assess convergence and data fit.

```{figure} ./images/mag_predicted.png
```

We note the following:

 - It took 5 iterations to converge to the target misfit.
 - The following 21 iterations have increased the model complexity ($phi_m$) while preserving the data fit ($phi_d$).
 - Small anomalies are under-fitted, which could warrant a second inversion with lower uncertainties.

#### Smooth solution (L2-norm)

```{figure} ./images/mag_l2_model.png
```

The smooth solution (iteration 5) indicates the presence of a susceptible body at depth. The shape and location of the anomaly resemble the solution obtained with the gravity modelling, including the presence of smaller "fuzzy" anomalies at the margins.  

#### Compact solution (L0-norm)

```{figure} ./images/mag_lp_model.png
```

As an alternative solution, the compact model (iteration 21) recovers a well-defined dense body within a mostly uniform background. Susceptibility contrasts have substantially increased in the range of [0.0, 2.0] SI.


(forrestania-joint-os)=
## Joint cross-gradient

After inverting the gravity and magnetic data independently, and recovering anomalies with similar shapes and positions in both, the next step is to invert both datasets jointly. We are going to employ the cross-gradient constraint to promote alignment of edges in the density and susceptibility models. 

### Inversion options

The following options are set for the inversion.

```{figure} ./images/joint_setup.png
---
scale: 10%
---
[Click to enlarge]
```

- Select the two existing inversion groups (magnetic and gravity) to Group A and B.
- Leave the mesh option empty. The routine will generate *an inversion mesh* based on the best resolution from both the gravity and magnetic problems.
- Set the topography

### Results

After completion of the inversion, we load the results

```{figure} ./images/joint_lp_model.png
```

We note the following:

- The shape of the density and magnetic susceptibility models match closely near the main anomaly.
- Some of the smaller anomalies, as well as an extension to the north-east, are only seen in the density model.

(forrestania-kmeans)=
## Analysis

The joint inversion results highlight a few features that could be distinguished by their physical property contrasts.
After running a K-means clustering on the final models, we recover the following simplified petrophysical model with 4 units.

```{figure} ./images/joint_kmeans.png
```

This table summarizes the different units based on their mean density and susceptibility values. 

```{table} Summary of expected physical properties
| Unit | Susceptibility | Density |
| :--- | :--- | :---- |
| background |  low    | low      |
| A |  high    | high      |
| B |  low    | high      |
| C |  high | low |
```

## Summary

- We have inverted publicly available data over the Forrestania geological province.

- The joint cross-gradient method was successfull in promoting similarities between the different physical properties wherever possible.

- Our results suggest the presence of different geological units with distinct density and magnetic susceptibility signatures.
